In [83]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
import random

In [79]:
def EA321_to_DCM(phi, theta, psi):

    sf = m.sin(phi)
    cf = m.cos(phi)
    st = m.sin(theta)
    ct = m.cos(theta)
    ss = m.sin(psi)
    cs = m.cos(psi)

    return np.array([[ct*cf, ct*sf, -st],
                    [ss*st*cf - cs*sf, ss*st*sf + cs*cf, ss*ct],
                    [ss*sf + cs*st*cf, -ss*cf + cs*st*sf, cs*ct]])

In [152]:
EA321 = np.array([30, 20, -10])*np.pi/180 #roll, pitch, yaw in radians
BN = EA321_to_DCM(EA321[0], EA321[1], EA321[2])

sN = np.array([1, 0, 0])
mN = np.array([0, 0, 1])

sB_true = BN@sN
mB_true = BN@mN

In [153]:
#adding noise

sB_observed = np.array([0, 0, 0])
mB_observed = np.array([0, 0, 0])

for i in range(3):
    sB_observed = sB_true + random.uniform(-0.01, 0.01)
    mB_observed = mB_true + random.uniform(-0.01, 0.01)

### TRIAD

In [154]:
t1_B = sB_observed
t2_B = np.cross(sB_observed, mB_observed)/np.linalg.norm(np.cross(sB_observed, mB_observed))
t3_B = np.cross(t1_B, t2_B)

TBbar = np.array([t1_B, t2_B, t3_B])

In [155]:
t1_N = sN
t2_N = np.cross(sN, mN)/np.linalg.norm(np.cross(sN, mN))
t3_N = np.cross(t1_N, t2_N)

TN = np.array([t1_N, t2_N, t3_N])

In [128]:
BNbar_TRIAD = TBbar.T@TN
print(BNbar_TRIAD)
print(BN)

[[ 0.80779509  0.48182174 -0.33319786]
 [-0.54984073  0.8197999  -0.15417327]
 [ 0.19887154  0.30947688  0.92715555]]
[[ 0.81379768  0.46984631 -0.34202014]
 [-0.54383814  0.82317294 -0.16317591]
 [ 0.20487413  0.31879578  0.92541658]]


### Davenport's q-method

In [147]:
def quat2dcm(q):
    """Convert quaternion to DCM.
    Parameters
    ----------
    q : numpy.array
        Quaternion in the form [q0 q1 q2 q3], where q0 is the rotation element.
    Returns
    -------
    dcm : numpy.array
        Corresponding direction cosine matrix.
    """
    return np.array(
        [
            [
                q[0] ** 2 + q[1] ** 2 - q[2] ** 2 - q[3] ** 2,
                2 * (q[1] * q[2] + q[0] * q[3]),
                2 * (q[1] * q[3] - q[0] * q[2]),
            ],
            [
                2 * (q[1] * q[2] - q[0] * q[3]),
                q[0] ** 2 - q[1] ** 2 + q[2] ** 2 - q[3] ** 2,
                2 * (q[2] * q[3] + q[0] * q[1]),
            ],
            [
                2 * (q[1] * q[3] + q[0] * q[2]),
                2 * (q[2] * q[3] - q[0] * q[1]),
                q[0] ** 2 - q[1] ** 2 - q[2] ** 2 + q[3] ** 2,
            ],
        ]
    )

In [149]:
v1_N = sN
v2_N = mN
v1_B = sB_observed
v2_B = mB_observed

w1 = 1
w2 = 1

In [159]:
B = w1*np.outer(v1_B, v1_N.T) + w2*np.outer(v2_B, v2_N.T)
sigma = np.trace(B)

S = B + B.T
S_K = S - sigma*np.eye(3)

Z = np.array([B[1][2] - B[2][1], B[2][0] - B[0][2], B[0][1] - B[1][0]])
Ztrans = Z.T

K = np.array([[sigma, Ztrans[0], Ztrans[1], Ztrans[2]],
             [Z[0], S_K[0][0], S_K[0][1], S_K[0][2]],
             [Z[1], S_K[1][0], S_K[1][1], S_K[1][2]],
             [Z[2], S_K[2][0], S_K[2][1], S_K[2][2]]])

In [164]:
eigvals, eigvecs = np.linalg.eig(K)

target = np.argmax(eigvals)

beta = eigvecs[:, target]

if beta[0] < 0:
    beta[0] *= -1

BNbar_qmethod = quat2dcm(beta)
print(BNbar_qmethod)

[[ 0.81021874  0.48182174 -0.33375652]
 [-0.55131267  0.8197999  -0.15486273]
 [ 0.19899733  0.30947688  0.92985167]]
